In [2]:
import cpp
import importlib
import pandas as pd
import numpy as np
from numba import njit,typeof,prange
from numba.typed import List

In [9]:
importlib.reload(cpp)

#@njit(parallel=True)
@njit()
def runcpp(cpp_out,rules_in,year,rage,cage):
    for i in range(1):
        count=0
        for y in year:
            rules=cpp.rules(rules_in,True)
            case = cpp.account(y,rules)
            for r in rage:
                for c in cage:
                #for j in prange(len(cage)):
                    #c = cage[j]
                    yr18=np.max(np.array([case.gYear(18),rules.start]))
                    nbre_con =case.gYear(r)-yr18
                    #case.SetHistory_ratio(r,{"r":1.5,'n':nbre_con})
                    #case.SetHistory_fam(claimage=c,age_birth=[])
                    ratiolist = [1.5 for x in range(nbre_con)]
                    case.RunCase(c,r,ratiolist)
                    ncontrib=case.ncontrib
                    ncontrib_s1=case.ncontrib_s1
                    ncontrib_s2=case.ncontrib_s2
                    ben=case.benefit
                    ben_s1=case.benefit_s1
                    ben_s2=case.benefit_s2
                    ampe = case.ampe
                    ampe_s1 = case.ampe_s1
                    ampe_s2 = case.ampe_s2
                    cpp_out['byear'][count]=y
                    cpp_out['retage'][count]=r
                    cpp_out['claimage'][count]=c
                    cpp_out['ncontrib'][count]=ncontrib
                    cpp_out['ncontrib_s1'][count]=ncontrib_s1
                    cpp_out['ncontrib_s2'][count]=ncontrib_s2
                    cpp_out['benefit'][count]=ben
                    cpp_out['benefit_s1'][count]=ben_s1
                    cpp_out['benefit_s2'][count]=ben_s2
                    cpp_out['ampe'][count]=ampe
                    cpp_out['ampe_s1'][count]=ampe_s1
                    cpp_out['ampe_s2'][count]=ampe_s2
                    count+=1
                    case.ResetCase()    


In [10]:
year=np.arange(1948,2000)
rage=np.arange(60,61)
cage=np.arange(60,65)
index = [x for x in range(len(year)*len(rage)*len(cage))]
cpp_out=pd.DataFrame(index=index,columns=['byear','retage','claimage','benefit','benefit_s1','benefit_s2','ncontrib','ncontrib_s1','ncontrib_s2','ampe', 'ampe_s1', 'ampe_s2']).fillna(0).to_records()
rules = cpp.load_rules(qpp=True)


In [11]:
%%timeit
runcpp(cpp_out,rules,year,rage,cage)

44.6 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
44.6/(len(cpp_out))


0.17153846153846156

In [19]:
cpp_out

rec.array([(  0, 1948, 60, 60,  619,    0,   0, 42,  0,  0,  3538,     0,    0),
           (  1, 1948, 60, 61,  674,    0,   0, 43,  0,  0,  3550,     0,    0),
           (  2, 1948, 60, 62,  731,    0,   0, 44,  0,  0,  3566,     0,    0),
           (  3, 1948, 60, 63,  788,    0,   0, 45,  0,  0,  3584,     0,    0),
           (  4, 1948, 60, 64,  846,    0,   0, 46,  0,  0,  3603,     0,    0),
           (  5, 1949, 60, 60,  636,    0,   0, 42,  0,  0,  3635,     0,    0),
           (  6, 1949, 60, 61,  693,    0,   0, 43,  0,  0,  3649,     0,    0),
           (  7, 1949, 60, 62,  751,    0,   0, 44,  0,  0,  3665,     0,    0),
           (  8, 1949, 60, 63,  810,    0,   0, 45,  0,  0,  3683,     0,    0),
           (  9, 1949, 60, 64,  868,    0,   0, 46,  0,  0,  3697,     0,    0),
           ( 10, 1950, 60, 60,  653,    0,   0, 42,  0,  0,  3736,     0,    0),
           ( 11, 1950, 60, 61,  712,    0,   0, 43,  0,  0,  3750,     0,    0),
           ( 12, 1950, 60, 6

In [60]:
case.ratio_list[41]

1.5

In [169]:
case.SetHistory(retage=65,a={"r":.5,'n':5})
case.RunCase(claimage=65)
case.benefit

42


18703.02841972143

In [181]:
nkids = 1
if nkids > 0 :
    print("yes")

yes


In [17]:
import numpy as np
import operator
from enum import Enum
import pandas as pd
from os import path

def load_rules(qpp=True):
    bnames = ['byear','era','nra','lra']
    ynames = ['year','ympe','exempt','worker','employer','selfemp','arf','drc','nympe','reprate',
        'droprate','pu1','pu2','pu3','pu4','survmax60', 'survmax65', 'survage1', 'survage2',
            'survrate1', 'survrate2','era','nra','lra','test','supp','disab_rate','disab_base','cola',
                'ympe_s2','worker_s1','employer_s1','worker_s2','employer_s2','selfemp_s1','selfemp_s2',
                'reprate_s1', 'reprate_s2']
    if (qpp==True):
        yrspars = pd.read_excel('./params/qpp_history.xlsx',names=ynames)
    else :
        yrspars = pd.read_excel('./params/cpp_history.xlsx',names=ynames)
    stop  = np.max(yrspars['year'].values)
    yrspars = yrspars.set_index('year')
    return yrspars.to_records()

In [55]:
pars = load_rules()

In [56]:
pars

rec.array([(1966,  5000.        ,  600, 0.018  , 0.018  , 0.036 , 0.   , 0.   , 3, 0.  , 0.15,   0.        ,   0.        ,   0.        ,   0.        ,     0.        ,     0.        , 45, 65, 0.375, 0.6, 67, 67, 70,   0.  , 0.   , 0.75,   0.        , 0.   , 1.  , 0.    , 0.    , 0.  , 0.  , 0.   , 0.  , 0.    , 0.    ),
           (1967,  5000.        ,  600, 0.018  , 0.018  , 0.036 , 0.   , 0.   , 3, 0.  , 0.15,   0.        ,   0.        ,   0.        ,   0.        ,   239.64      ,   239.64      , 45, 65, 0.375, 0.6, 67, 67, 70,   0.  , 0.   , 0.75,   0.        , 0.   , 1.  , 0.    , 0.    , 0.  , 0.  , 0.   , 0.  , 0.    , 0.    ),
           (1968,  5100.        ,  600, 0.018  , 0.018  , 0.036 , 0.   , 0.   , 3, 0.25, 0.15,  25.5       ,  25.5       ,  25.5       ,  25.5       ,   366.96      ,   366.96      , 45, 65, 0.375, 0.6, 67, 67, 70,   0.  , 0.   , 0.75,   0.        , 0.02 , 1.  , 0.    , 0.    , 0.  , 0.  , 0.   , 0.  , 0.    , 0.    ),
           (1969,  5200.        ,  60

In [65]:
np.where(pars['year']==1980,pars['year'],0).argmax()

14

In [63]:
pars["worker"][np.where(year==1980,year,0).argmax()]

0.018

In [60]:
year

array([1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976,
       1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
       1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998,
       1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024, 2025])